The data started in this "untidy" format split across two files.

The goal of this notebook is to make it so it's a single file, where each row is a single stop, and each possible attribute/result of the stop is a binary categorical variable.

The output is stored in a CSV in the data directory.

In [ ]:
import pandas as pd

df = pd.read_csv('../data/police_stop_data_main_2018.csv')
df_results = pd.read_csv('../data/police_stop_data_results_2018.csv')

In [2]:
df.shape, df_results.shape

((8215, 12), (39569, 5))

In [27]:
df['raceethn'] = df['race'] + df['ethnic']

In [28]:
df_results['appkey-item'] = df_results['appkey'] + '-' + df_results['itemdesc']

In [5]:
df_results.head()

,appkey,appid,itemcode,itemdesc,addtime,appkey-item
0,RPT1,1.0,VEH,VEHICLE,9/26/2018 15:09,RPT1-VEHICLE
1,RPT1,2862.0,VEH,VEHICLE,1/1/2018 16:26,RPT1-VEHICLE
2,RPT2,2862.0,SPEE,TRAFFIC-SPEEDING,1/1/2018 16:26,RPT2-TRAFFIC-SPEEDING
3,RPT3,2862.0,*NO,*NO SEARCH,1/1/2018 16:26,RPT3-*NO SEARCH
4,RPT6,2862.0,WAR,WARNING,1/1/2018 16:26,RPT6-WARNING


In [29]:
dummies = pd.get_dummies(df_results['appkey-item'])

In [30]:
df_results_long = pd.concat([df_results, dummies], axis=1)

In [14]:
df_results_long.head()

,appkey,appid,itemcode,itemdesc,addtime,appkey-item,RPT1-PEDESTRIAN,RPT1-VEHICLE,RPT2-DISTURBANCE,RPT2-EQUIPMENT VIOLATION,...,RPT6-FIELD CONTACT,RPT6-M-1 HOLD,RPT6-NO ACTION,RPT6-WARNING,RPT7-FELONY,RPT7-MISDEMEANOR,RPT7-MUNICIPAL,RPT7-NOT CHARGED,RPT7-TRAFFIC,RPT7-WARRANT
0,RPT1,1.0,VEH,VEHICLE,9/26/2018 15:09,RPT1-VEHICLE,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,RPT1,2862.0,VEH,VEHICLE,1/1/2018 16:26,RPT1-VEHICLE,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,RPT2,2862.0,SPEE,TRAFFIC-SPEEDING,1/1/2018 16:26,RPT2-TRAFFIC-SPEEDING,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,RPT3,2862.0,*NO,*NO SEARCH,1/1/2018 16:26,RPT3-*NO SEARCH,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,RPT6,2862.0,WAR,WARNING,1/1/2018 16:26,RPT6-WARNING,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [31]:
df_results_tidy_by_stop = df_results_long.groupby('appid').sum()

In [32]:
df_results_tidy_by_stop.shape

(8208, 47)

In [33]:
df.shape

(8215, 13)

In [3]:
df.head()

,stopdate,stoptime,streetnbr,streetdir,street,Min,sex,race,ethnic,Year of birth,enfaction,rpmainid
0,01/01/2018,1619,4561,NaN,ARAPAHOE AVE/MACARTHUR DR,5,M,W,N,1976,Y,2862
1,01/01/2018,1630,6600,NaN,ARAPAHOE AVE,10,M,W,N,2000,N,2863
2,12/31/2017,214,1799,NaN,BROADWAY/CANYON BLVD,35,M,W,N,1991,N,2864
3,12/31/2017,214,1799,NaN,BROADWAY/CANYON BLVD,35,M,W,N,1992,N,2865
4,12/31/2017,214,1799,NaN,BROADWAY/CANYON BLVD,350,M,W,N,1958,N,2866


In [34]:
df_merged = pd.merge(df, df_results_tidy_by_stop, left_on='rpmainid', right_on='appid')

In [35]:
df_merged.to_csv('../data/tidy.csv', index=False)